In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

# Resend alerts on tickets that remain OPEN or in WORKING state for TOO LONG
print("============================================")
print("/  ReissueTicketAlert is running.               /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']

  _TICKETAGE_REISSUE_THRESHOLD_SECONDS = int(config['analytics']['ticketreissueseconds'])
  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  ReissueTicketAlert is running.               /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
def postMessageToSlackInfrastructure(text):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, text=text, blocks=[])
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(response)
    else:
      print("Ok posting message to Slack channel")   

In [3]:
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 300

#Ticket Status
_TICKET_STATUS_OPEN    = 10000
_TICKET_STATUS_WORKING = 10001

_PREFERRED_IMAGE = "https://www.dropbox.com/s/ndxejw1xfd0x8z3/alert-icon.jpg?raw=1"

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 5)

ticketquery = "SELECT * FROM kanji_ticket WHERE status_id={} OR status_id={}".format(_TICKET_STATUS_OPEN, _TICKET_STATUS_OPEN)
logger(_LOG_INFO, ticketquery)
df = pd.read_sql(ticketquery, conn)

logger(_LOG_DEBUG, "number of tickets OPEN or WORKING {}".format(len(df.index))) 
for ind in df.index:
    idticket = df['idticket'][ind]
    ticketts = df['ticketts'][ind]
    ticketstatus = df['status_id'][ind]
    ticketopentime = df['opentimestamp'][ind]
    description = df['description'][ind]
    ticketchanneldbid = df['slackticketchannel_id'][ind]
    nodeid = df['node_id'][ind]
    nodequery = "SELECT node.name, location.description, location.imageurl, \
                 customer.slacktoken FROM kanji_node node \
                 JOIN kanji_location location ON node.location_id=location.idlocation \
                 JOIN kanji_customer customer ON node.customer_id=customer.idcustomer \
                 WHERE idnode={}".format(nodeid)
    df2 = pd.read_sql(nodequery, conn)
    nodename = df2["name"][0]
    locationdescription = df2["description"][0]
    _SLACK_TOKEN = df2["slacktoken"][0]
    
    channelQuery = "SELECT * FROM kanji_slackchannel WHERE idslackchannel={}".format(ticketchanneldbid)
    df3 = pd.read_sql(channelQuery, conn)
    _SLACK_CHANNEL_ID = df3['channelid'][0]
    
    ticketage = ticketnow - ticketopentime  #df['lastupdatetimestamp'][ind]
    ticketopenage = ticketnow - ticketopentime
    ticketageseconds =  ticketage.days*24*60*60 + ticketage.seconds
    logger(_LOG_INFO, "ticket={} is {} old".format(idticket, ticketage))
    if ticketageseconds >= _TICKETAGE_REISSUE_THRESHOLD_SECONDS:
      logger(_LOG_DEBUG, "")
      logger(_LOG_DEBUG, nodename)
      logger(_LOG_DEBUG, locationimageurl)
      logger(_LOG_DEBUG, _SLACK_CHANNEL_ID)
      logger(_LOG_DEBUG, idticket )
      mentions = "@Charlie"
      message = description
      postMessageToSlackInfrastructure("ticket {} is {} old.".format(idticket, ticketage))
      ts = kt.slackreissueticket(nodename, locationdescription, message, mentions, 2, 3, \
                        locationimageurl, _SLACK_TOKEN, _SLACK_CHANNEL_ID, idticket, ticketts, ticketstatus, ticketopenage, ticketopentime)
      if ts!=0:
        kt.updateTicket(conn, idticket, ticketts)
        logger(_LOG_INFO, "Alert reissued for ticket {}.".format(idticket))
    else:
      status = "Ok"
      #is there an OPEN ticket on this mote?
    
logger(_LOG_INFO, "ReissueTicketAlert Done!")  

SELECT * FROM kanji_ticket WHERE status_id=10000 OR status_id=10000
ReissueTicketAlert Done!
